# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet') # download for lemmatization
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer , TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kharuto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kharuto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kharuto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disasterpipe.db')
df = pd.read_sql_table('dp.messages', engine)

In [3]:
# could also think about including message type
x_cols = 'message'
X = df[x_cols].astype(str) #would like to use original too, but idk which language it is... some creol?
Y = df.drop(columns = [x_cols, 'id', 'original', 'genre'])

In [4]:
Y.related.unique()

array([1, 0])

In [5]:
lf = lambda text : re.sub(r'[^a-zA-Z0-9]', ' ')

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
#     normalize
    text = re.sub(r'[^a-zA-z0-9]',' ', text.lower())
    
    words = [word for word in word_tokenize(text) if word not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in words:
        cleanboi = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(cleanboi)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('textpipe', CountVectorizer(tokenizer = tokenize)), # put TFIDF here if necessary
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs = -1)))
    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [11]:
model = pipeline
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [12]:
y_test.related.unique()

array([0, 1])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
# https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin
print(classification_report(y_test.values, y_pred, target_names=Y.columns.values))


                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      1962
               request       0.80      0.51      0.62       428
                 offer       0.00      0.00      0.00        14
           aid_related       0.76      0.71      0.73      1045
          medical_help       0.81      0.06      0.11       224
      medical_products       0.80      0.08      0.15       142
     search_and_rescue       0.50      0.03      0.05        80
              security       0.00      0.00      0.00        58
              military       0.75      0.04      0.07        82
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.33      0.48       171
                  food       0.77      0.69      0.73       275
               shelter       0.84      0.40      0.54       243
              clothing       0.88      0.19      0.32        36
                 money       1.00      

/home/kharuto/Documents/projects/b-kaindl/udacity_new/lesson2/project/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kharuto/Documents/projects/b-kaindl/udacity_new/lesson2/project/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kharuto/Documents/projects/b-kaindl/udacity_new/lesson2/project/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted label

### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done 205 tasks      | elapsed: 38.7min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 67.0min finished


In [29]:
parameters = {'clf__estimator__criterion': ['gini', 'entropy'],
'clf__estimator__max_depth' : [125,150,175],
'clf__estimator__n_estimators' : [1,5],
'clf__estimator__min_samples_leaf' : [5,10]}
    
cv = GridSearchCV(pipeline, param_grid=parameters, cv = 3, verbose=4, n_jobs =-1, scoring = 'f1_weighted') 
cv.fit(X_train, y_train)
y_pred_cv = cv.predict(X_test)
print(cv.best_params_)
print(classification_report(y_test.values, y_pred_cv, target_names=Y.columns.values))

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed: 12.6min finished


{'clf__estimator__criterion': 'gini', 'clf__estimator__max_depth': 125, 'clf__estimator__min_samples_leaf': 5, 'clf__estimator__n_estimators': 1}
                        precision    recall  f1-score   support

               related       0.79      0.93      0.86      5941
               request       0.54      0.39      0.45      1333
                 offer       0.00      0.00      0.00        34
           aid_related       0.63      0.64      0.64      3286
          medical_help       0.42      0.08      0.14       644
      medical_products       0.24      0.02      0.04       414
     search_and_rescue       0.36      0.05      0.09       239
              security       0.00      0.00      0.00       156
              military       0.59      0.11      0.19       267
           child_alone       0.00      0.00      0.00         0
                 water       0.43      0.10      0.16       512
                  food       0.59      0.18      0.28       878
               shelte

/home/kharuto/Documents/projects/b-kaindl/udacity_new/lesson2/project/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kharuto/Documents/projects/b-kaindl/udacity_new/lesson2/project/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kharuto/Documents/projects/b-kaindl/udacity_new/lesson2/project/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted label

### 9. Export your model as a pickle file

In [16]:
import pickle
with open("model_75acc.pkl", 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.